In [22]:
import plotly.express as px
from database_utils import *

iteration_count = 10 

In [2]:
# Read the testing results from the snowflake table
connect_to_snowflake()
results = read_from_database('FINCOPILOT_QA.DEV_TESTING.FINAL_OUTPUT_ITERATIONS')
prompts = read_from_database('FINCOPILOT_QA.DEV_TESTING.MASTER_USER_PROMPTS')
prompts.index = prompts['PROMPT_ID']
prompts.drop(columns=['PROMPT_ID'], inplace=True)

disconnect_from_snowflake()

In [7]:
results

,PROMPT_TEMPLATE_ID,PROMPT_ID,ITER_NUM,RUN_DATE_TIME,GOLDEN_SQL,LLM_GEN_SQL,LLM_GEN_SQL_EXECUTABLE,GOLDEN_COLS,LLM_GEN_COLS,GOLDEN_ROW_COUNT,LLM_ROW_COUNT,GOLDEN_COLUMN_COUNT,LLM_COLUMN_COUNT,ROW_COUNT_MATCHED,COL_COUNT_MATCHED,DATA_MATCHED,COLS_IN_GOLDEN_NOT_IN_LLM,COMMON_COLUMNS,GENERATED_DATE
0,9,PRMT_00009,1,20240912164948,SELECT total_overdue_amount FROM (SELECT ds....,SELECT total_overdue_amount FROM (SELECT ...,True,"subsidiary_name,total_overdue_amount",total_overdue_amount,0,0,2,1,True,False,True,subsidiary_name,total_overdue_amount,2024-09-12
1,9,PRMT_00009,2,20240912164948,SELECT total_overdue_amount FROM (SELECT ds....,SELECT total_overdue_amount FROM (SELECT ...,True,"subsidiary_name,total_overdue_amount",total_overdue_amount,0,0,2,1,True,False,True,subsidiary_name,total_overdue_amount,2024-09-12
2,9,PRMT_00009,3,20240912164948,SELECT total_overdue_amount FROM (SELECT ds....,SELECT total_overdue_amount FROM (SELECT ...,True,"subsidiary_name,total_overdue_amount",total_overdue_amount,0,0,2,1,True,False,True,subsidiary_name,total_overdue_amount,2024-09-12
3,9,PRMT_00009,4,20240912164948,SELECT total_overdue_amount FROM (SELECT ds....,SELECT total_overdue_amount FROM (SELECT ...,True,"subsidiary_name,total_overdue_amount",total_overdue_amount,0,0,2,1,True,False,True,subsidiary_name,total_overdue_amount,2024-09-12
4,9,PRMT_00009,5,20240912164948,SELECT total_overdue_amount FROM (SELECT ds....,SELECT total_overdue_amount FROM (SELECT ...,True,"subsidiary_name,total_overdue_amount",total_overdue_amount,0,0,2,1,True,False,True,subsidiary_name,total_overdue_amount,2024-09-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,235,PRMT_00235,6,20240912164948,SELECT total_invoices FROM (SELECT COUNT(...,SELECT total_invoices FROM (SELECT COUNT(...,True,total_invoices,total_invoices,0,0,1,1,True,True,True,None,total_invoices,2024-09-12
406,235,PRMT_00235,7,20240912164948,SELECT total_invoices FROM (SELECT COUNT(...,SELECT total_invoices FROM (SELECT COUNT(...,True,total_invoices,total_invoices,0,0,1,1,True,True,True,None,total_invoices,2024-09-12
407,235,PRMT_00235,8,20240912164948,SELECT total_invoices FROM (SELECT COUNT(...,SELECT total_invoices FROM (WITH last_month AS...,True,total_invoices,total_invoices,0,0,1,1,True,True,True,None,total_invoices,2024-09-12
408,235,PRMT_00235,9,20240912164948,SELECT total_invoices FROM (SELECT COUNT(...,SELECT total_invoices FROM (SELECT COUNT(...,True,total_invoices,total_invoices,0,0,1,1,True,True,True,None,total_invoices,2024-09-12


In [28]:
groupd =results.groupby('PROMPT_ID')[['ROW_COUNT_MATCHED', 'COL_COUNT_MATCHED', 'DATA_MATCHED']].sum().reset_index()

groupd.index = groupd['PROMPT_ID']
groupd.drop(columns=['PROMPT_ID'], inplace=True)
groupd


sql_outputs = results.groupby('PROMPT_ID')[[ "LLM_GEN_SQL", "GOLDEN_SQL", "DATA_MATCHED"]].apply(lambda x: x[x['DATA_MATCHED'] != iteration_count].head(1)).reset_index()
sql_outputs.index = sql_outputs['PROMPT_ID']
sql_outputs.drop(columns=['PROMPT_ID'], inplace=True)
sql_outputs


groupd = groupd.merge(sql_outputs, on='PROMPT_ID', how='left')
groupd.drop(columns=['DATA_MATCHED_y'], inplace=True)
groupd.rename(columns={'DATA_MATCHED_x': 'DATA_MATCHED'}, inplace=True)
groupd.drop(columns='level_1', inplace=True)
groupd

,ROW_COUNT_MATCHED,COL_COUNT_MATCHED,DATA_MATCHED,LLM_GEN_SQL,GOLDEN_SQL
PROMPT_ID,,,,,
PRMT_00009,10,0,10,SELECT total_overdue_amount FROM (SELECT ...,SELECT total_overdue_amount FROM (SELECT ds....
PRMT_00010,10,10,10,"SELECT brand, total_invoiced_amount FROM (SELE...","SELECT brand, total_invoiced_amount FROM (SELE..."
PRMT_00011,10,0,10,"SELECT invoice_due_amount, invoice_due_date, i...","SELECT invoice_due_amount, invoice_due_date, i..."
PRMT_00013,1,1,1,"SELECT invoice_aging_bucket, subsidiary_name F...","SELECT invoice_aging_bucket, subsidiary_name F..."
PRMT_00016,10,0,10,SELECT total_overdue_amount FROM (SELECT ...,SELECT total_overdue_amount FROM (WITH last_qu...
PRMT_00017,10,1,9,"SELECT ai.INVOICE_NUMBER, ai.INVOICE_...","SELECT ac.customer_name, ai.INVOICE_NUMBER..."
PRMT_00020,10,3,10,"SELECT invoice_aging_bucket, invoice_due_amoun...","SELECT invoice_aging_bucket, invoice_due_amoun..."
PRMT_00021,10,4,10,"SELECT invoice_aging_in_days, invoice_due_amou...","SELECT invoice_aging_in_days, invoice_due_amou..."
PRMT_00027,10,10,10,SELECT total_payments_received FROM (SELECT ...,SELECT total_payments_received FROM (SELECT SU...


In [35]:
delinquents = groupd[groupd['DATA_MATCHED'] != 10]
delinquents = delinquents.merge(prompts, on='PROMPT_ID', how='left')
fig = px.bar(delinquents, x=delinquents.index, y='DATA_MATCHED',title='Data Matched', hover_data=['LLM_GEN_SQL'])

fig

In [34]:
delinquents.to_csv('delinquent_prompts.csv')

In [38]:
prompts[prompts.index == 'PRMT_00098']

,PROMPT_TEMPLATE_ID,PROMPT
PROMPT_ID,,
PRMT_00098,98,list all open invoices for dris ticket# 1054
